Projekt Vizualizácia
---

Téma: Psy

Skupina Číslo: 10
Skupina Názov: Gormiti

Členovia: Adam Mjartan, Martin Bakus, Norbert Riebesam, Timotej Vida, Michal Salčík

Použité Datasety (linky):

- Hlavný Dataset: https://figshare.com/ndownloader/files/27715521
- Geografické Dáta: https://worldostats.com/country-stats/dog-population-by-country/
- Populačné  Dáta: https://www.kaggle.com/datasets/dataanalyst001/world-population-by-country-2024


Dodatočné Zdroje:



Import Použitých Knižníc

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly
import geopandas as gpd
import plotly.graph_objects as go
import numpy as np
from IPython.display import Markdown
from matplotlib.patches import Patch

Načítanie Použitých Datasetov

In [ ]:
# main_data = pd.read_excel(r'https://figshare.com/ndownloader/files/27715521', sheet_name='Dataset')
# dogs = main_data
# main_data_information = pd.read_excel(r'https://figshare.com/ndownloader/files/27715521', sheet_name='Information')

main_data = pd.read_csv("https://raw.githubusercontent.com/Retr-1/AnalyzaPsy/refs/heads/master/psy_data.csv", header=0)
dogs = main_data
main_data_information = pd.read_csv("https://raw.githubusercontent.com/Retr-1/AnalyzaPsy/refs/heads/master/psy_info.csv", header=0)

population_data = pd.read_csv(r"https://raw.githubusercontent.com/P3rfectS/Data/refs/heads/main/World%20Population%20by%20country%202024.csv")
#geografical_data = pd.read_csv(r"https://raw.githubusercontent.com/P3rfectS/Data/refs/heads/main/Dog%20Population%20by%20Country.csv")
geografical_data = pd.read_csv("https://raw.githubusercontent.com/Retr-1/AnalyzaPsy/a83cd851e4664a59dcd5c589cee2720b9aec8b59/Misov%20Priestor/data/Dog%20Population%20by%20Country.csv")
world_geography = gpd.read_file(r"https://github.com/P3rfectS/Data/raw/refs/heads/main/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp")

Príprava Vizuálnych Podkladov
---

Príprava: Geografické Dáta
---

#1 transofmácia, čistenie a doplnenie datasetu

In [ ]:
#Odstránenie chybných dát
geografical_data = geografical_data[geografical_data["Country"] != 'Montenegro']

#Doplnenie chýbajúcich dát (tvorba dočasného df)
#new_row = pd.DataFrame([{"Country": "Australia", "Estimated Dog Population": "6000000"}])

#Pripojenie ku pôvodnému datasetu
#geografical_data = pd.concat([geografical_data, new_row], ignore_index=True)


#Zmena typu dát na numerické
geografical_data['Estimated Dog Population'] = pd.to_numeric(
    geografical_data['Estimated Dog Population'].str.replace(',', ''),
    errors='coerce'
)

#Úprava názvov štátov v datasete aby zodpovedali názvom ktoré používa funkcia
country_name_fixes = {
    "Swaziland": "Eswatini",
    "Democratic Republic of Congo": "DR Congo",
    "Kosovo": "Kosovo",
    "United States of America": "United States",
    "Czech Republic": "Czechia",
    "United States": "United States of America",
    "Bosnia and Herzegovina": "Bosnia and Herz."
}
geografical_data['Country'] = geografical_data['Country'].replace(country_name_fixes)
population_data['Country'] = population_data['Country'].replace(country_name_fixes)

#2 Tvorba zjednotených tabuľiek

In [ ]:
#Tvorba zjednotenej tabulky
full_table = geografical_data.merge(population_data, on='Country', how='left')
full_table['Dog Population per Capita'] = full_table['Estimated Dog Population'] / full_table['Population 2024']
full_table["Area"] = pd.to_numeric(full_table['Area (km2)'].str.replace(r'K', 'e3', regex=True).str.replace(r'M', 'e6', regex=True))
full_table["Dog Population Density"] = full_table["Estimated Dog Population"] / full_table["Area"]
#Tvorba tabuliek pre vizualizácie
merged = world_geography.merge(geografical_data, how='left', left_on='NAME', right_on='Country')
merged["Estimated Dog Population log"] = np.log10(merged["Estimated Dog Population"])
merged_per_capita = world_geography.merge(full_table, how='left', left_on='NAME', right_on='Country')
merged_per_capita["Dog Population Density log"] = np.log10(merged_per_capita["Dog Population Density"])

#3 Tvorba jednotlivých vizualizácií

In [ ]:
#Funkcia pre vytvorenie geografickej vizualizácie psej populácie v rôznych krajinách sveta
def geo_vytvor_populacia_psov():
    fig, ax = plt.subplots(figsize=(15, 6))
    mapa = merged.plot(
        column='Estimated Dog Population log',
        ax=ax,
        legend=True,
        cmap='OrRd',
        missing_kwds={"color": "lightgrey", "label": "No data"}
    )
    cbar = mapa.get_figure().get_axes()[-1]
    tick_values = cbar.get_yticks()
    cbar.set_yticklabels([f"10^{val:,}" for val in tick_values])

    plt.title("Estimated Dog Population by Country (log10)")
    plt.axis('off')
    plt.show()

#Funkcia pre vytvorenie geografickej vizualizácie populácie psov na jedného obyvateľa v rôznych krajinách sveta
def geo_vytvor_psy_na_hlavu():
    fig, ax = plt.subplots(figsize=(15, 6))
    merged_per_capita.plot(
    column='Dog Population per Capita',
    ax=ax,
    legend=True,
    cmap='OrRd',
    missing_kwds={"color": "lightgrey", "label": "No data"}
    )
    plt.title("Dog Population per Capita")
    plt.axis('off')
    plt.show()

def geo_vytvor_hustotu_psov():
    fig, ax = plt.subplots(figsize=(15, 6))
    mapa = merged_per_capita.plot(
    column='Dog Population Density log',
    ax=ax,
    legend=True,
    cmap='OrRd',
    missing_kwds={"color": "lightgrey", "label": "No data"}
    )

    cbar = mapa.get_figure().get_axes()[-1]
    tick_values = cbar.get_yticks()
    cbar.set_yticklabels([f"10^{val:,}" for val in tick_values])

    plt.title("Dog population density (per km^2, log10)")
    plt.axis('off')
    plt.show()

    
geo_vytvor_populacia_psov()
geo_vytvor_psy_na_hlavu()
geo_vytvor_hustotu_psov()
#HUSTOTA NA KM4

Práca s pôvodnými dátami
---

In [ ]:
sns.histplot(data = main_data, x = "daily_exercise")
figure, axes = plt.subplots(2,2, figsize = (12,10))
for i, n in enumerate(["barking_score","stranger_aggression_score","owner_aggression_score","dog_aggression_score"]):
    sns.violinplot(data = main_data, x = "impulsive", y = n, ax = axes[i//2][i%2], color = "orange")
    axes[i//2][i%2].set_title("How impulsivity influences " + n.replace("_", " ").title())

empathic_table = main_data[main_data['Empathic'].notna()].copy(deep = True)
empathic_means = pd.DataFrame(empathic_table.groupby("breed_group")["Empathic"].mean())
empathic_means = empathic_means.sort_values("Empathic")
fig = px.bar(empathic_means, y = empathic_means.index, x='Empathic', title = "Empathic by dog breed")
fig.update_layout(
    yaxis=dict(tickmode='linear'),  # Force all ticks to show
    width=1200,
    height=800,
    bargap=0.05
)
fig.show()

figure2, axes2 = plt.subplots(3,1, figsize = (6,15))
sns.violinplot(data = main_data, x = "aggression_strangers", y = "stranger_aggression_score", ax = axes2[0], color = "skyblue")
sns.violinplot(data = main_data, x = "aggression_owner", y = "owner_aggression_score", ax = axes2[1], color = "skyblue")
sns.violinplot(data = main_data, x = "aggression_dogs", y = "dog_aggression_score", ax = axes2[2], color = "skyblue")
for i in range(3):
    axes2[i%3].set_title("How accurate are the owners answers compared to the calculated score")
figure2.subplots_adjust(hspace=0.4)

In [ ]:
figure, axes = plt.subplots(2, 2, figsize = (12, 10))

axes[0][0].hist2d(main_data["human_sociability_score"].dropna(), main_data["dog_sociability_score"].dropna(), bins = 20)
axes[0][0].set_title("Dog sociability (relation between sociability to humans and dogs)")
axes[0][0].set_xlabel("Human sociability score")
axes[0][0].set_ylabel("Dog sociabilty score")

axes[0][1].hist2d(main_data["insecurity_score"].dropna(), main_data["aggressiveness_dominance_score"].dropna(), bins = 20)
axes[0][1].set_title("Relation between insecurity and aggressiveness of dogs)")
axes[0][1].set_xlabel("Insecurity score")
axes[0][1].set_ylabel("Aggressiveness dominance score")

axes[1][0].hist2d(main_data["human_sociability_score"].dropna(), main_data["aggressiveness_dominance_score"].dropna(), bins = 20)
axes[1][0].set_title("Relation between sociability to humans and aggressiveness)")
axes[1][0].set_xlabel("Human sociability score")
axes[1][0].set_ylabel("Aggressiveness dominance score")

axes[1][1].hist2d(main_data["activity_playfulness_score"].dropna(), main_data["dog_sociability_score"].dropna(), bins = 20)
axes[1][1].set_title("Relation between playfulness and dog sociability)")
axes[1][1].set_xlabel("Activity playfulness score")
axes[1][1].set_ylabel("Dog sociability score")
figure.subplots_adjust(wspace=0.4)



figure2, axes2 = plt.subplots(2,2, figsize = (12, 10))
sns.kdeplot(x = main_data["human_sociability_score"].dropna(), y = main_data["dog_sociability_score"].dropna(), ax = axes2[0][0], fill = True)
axes2[0][0].set_title("Dog sociability (relation between sociability to humans and dogs)")
axes2[0][0].set_xlabel("Human sociability score")
axes2[0][0].set_ylabel("Dog sociabilty score")

sns.kdeplot(x = main_data["insecurity_score"].dropna(), y = main_data["aggressiveness_dominance_score"].dropna(), ax = axes2[0][1], fill = True)
axes2[0][1].set_title("Relation between insecurity and aggressiveness of dogs)")
axes2[0][1].set_xlabel("Insecurity score")
axes2[0][1].set_ylabel("Aggressiveness dominance score")

sns.kdeplot(x = main_data["human_sociability_score"].dropna(), y = main_data["aggressiveness_dominance_score"].dropna(), ax = axes2[1][0], fill = True)
axes2[1][0].set_title("Relation between sociability to humans and aggressiveness)")
axes2[1][0].set_xlabel("Human sociability score")
axes2[1][0].set_ylabel("Aggressiveness dominance score")

sns.kdeplot(x = main_data["activity_playfulness_score"].dropna(), y = main_data["dog_sociability_score"].dropna(), ax = axes2[1][1], fill = True)
axes2[1][1].set_title("Relation between playfulness and dog sociability)")
axes2[1][1].set_xlabel("Activity playfulness score")
axes2[1][1].set_ylabel("Dog sociability score")
figure.subplots_adjust(hspace=0.4)
plt.show()

In [ ]:

figure, axes = plt.subplots(2,3, figsize = (18,15))
for i, n in enumerate(["metal_grid", "shiny_floor", "open_stairs", "closed_stairs", "moving_surface", "slippery_floor"]):
    sns.barplot(data = main_data, x = "fear_surfaces", y = n, ax = axes[i//3][i%3], color = "slateblue")
    axes[i//3][i%3].set_ylim(bottom = 1)
    axes[i//3][i%3].set_title(n.replace("_", " ").title() + " by answer for fear surfaces")

Príprava: Predstavenie Datasetu // Dataset Bias
---

In [ ]:
fig = px.bar(dogs['breed_group'].value_counts().reset_index(), y='breed_group', x='count', title='Pocty jednotlivych plemien v datach', width=1000, height=700, )
fig.update_layout(
    yaxis=dict(tickmode='linear'),  # Force all ticks to show
    width=1200,
    height=800,
    bargap=0.05
)
fig.show()

In [ ]:
df = dogs.groupby(by=['sterilization_status','sex'])['dog.id'].count()#.rename(columns={'dog.id':'count'})#.reset_index().set_index(['sex', 'sterilization_status'])

fig = go.Figure(data=[
    go.Bar(name='sterlized', x=['male', 'female'], y=[df['sterilized', 'male'], df['sterilized', 'female']]),
    go.Bar(name='intact', x=['male', 'female'], y=[df['intact', 'male'], df['intact', 'female']])
])
fig.update_layout(barmode='stack', width=700, title='Number of male and female sterilized/intact dogs')
fig.show()

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Histogram(x=dogs.query('sex == "male"')['weight_kg'], name='Male', hovertext=dogs.query('sex == "male"')['dog.id']))
# fig.add_trace(go.Histogram(x=dogs.query('sex == "female"')['weight_kg'], name='Female',  hovertext=dogs.query('sex == "female"')['dog.id']))

# # Overlay both histograms
# fig.update_layout(barmode='overlay', xaxis_title='Weight in KG', title='Weight Histogram by gender')
# # Reduce opacity to see both histograms
# fig.update_traces(opacity=0.75)
# fig.show()
sns.histplot(dogs, x='weight_kg', hue='sex', common_norm=False, stat='probability',hue_order=['female', 'male'], palette=['C3', 'C0'],)
plt.xlabel('Weight [kg]')
pass

In [ ]:
categories = ['Noise.sensitivity_age',
'Fearfulness_age',
'Aggression_age',
'Surface_age',
'Separation_age',
'Impulsivity_age',
'Personality_age']

ages_melt = dogs[categories + ['dog.id']].melt(id_vars='dog.id')

sns.kdeplot(data=ages_melt, x='value', hue='variable')
plt.title('KDE plot of ages at which the forms were answered')
pass

In [ ]:
categories = ['Noise.sensitivity_age',
'Fearfulness_age',
'Aggression_age',
'Surface_age',
'Separation_age',
'Impulsivity_age',
'Personality_age']
A = dogs[categories]
B = A[(A.nunique(axis=1) == 1)]
# display(Markdown('<h2>Vsetky riadky, ktore nemaju ani jednu hodnotu NaN a maju rozdielne veky vyplnenia formularov</h2>'))
display(Markdown(f'<h2>{B.shape[0]} záznamov má všetky dotazníky vyplnené v rovnakom veku, alebo nevyplnené</h2>'))
display(B)


In [ ]:
A = dogs[categories]
# B = pd.Series([1,1]) / pd.Series([0, 0]) - pd.Series([1,1]) / pd.Series([0,0]) # Creates NaN series
A = A[A.nunique(axis=1) > 1]
A['delta'] = A.max(axis=1) - A.min(axis=1)
A = A.sort_values(by='delta', ascending=False)
A['mean_age'] = A[categories].mean(axis=1)
display(Markdown('<h2>V niektorych pripadoch su rozdiely az 1 az 2 roky, co by mohlo sposobit mensie vychylky v konzistentnosti dat'))
display(A.head(10))
B = A.iloc[1:, ]
# sns.stripplot(A['delta'], alpha=0.3)
# plt.ylim([0,2.5])
sns.displot(B['delta'], kind='ecdf')
plt.xlabel('Rozdiel medzi min a max vekom')
plt.grid(visible=True, which='both', linestyle='--', alpha=0.7)
plt.title("ECDF rozdielov vekov, v ktorých boli vyplnené formuláre")
plt.xticks(np.linspace(start=0, stop=2, num=9).round(2))
plt.yticks(np.linspace(start=0, stop=1, num=9).round(2))
# sns.displot(B['delta'], kind='ecdf')

plt.subplots()
sns.scatterplot(B, x='mean_age', y='delta', alpha=0.3)
plt.title("Rozdiel vo vekoch psa")

display(Markdown(f'<h2>Počet psov, ktoré majú rozdiel väčší ako 1 a vek menší ako 2 roky = {B.query('delta > 1 and mean_age < 2').shape[0]}'))
pass

In [ ]:
features = """
Willing_to_learn
Playful_dogs
Playful_people
Obedient
Anxious
Lazy
Intelligent
Reliable
Human_dependent
Erratic
Stubborn
Fearful_people
Fearful_dogs
Curious
Impulsive
Affectionate_people
Persevering
Distractible
Submissive
Cautious
Dominant
Patient
Sociable_dogs_samegender
Sociable_dogs_oppositegender
Human_oriented
Prey_driven_search
Empathic
Aggressive_dogs_samegender
Greedy
Attention_seeking
Indifferent
Aggressive_people
Affectionate_dogs
"""
features = features.split()
A = dogs[features].dropna().transpose()
sns.clustermap(A, xticklabels=False, cmap='YlOrBr', figsize=(15,10))

In [ ]:
B = A.transpose().query("Aggressive_people >= 4").transpose()
display(Markdown("<h1>Dogs with Aggressive_people >= 4</h1>"))
print('Dog count:', B.shape[1])
sns.clustermap(B, xticklabels=False, cmap='YlOrBr', figsize=(15,10))

In [ ]:
CORR = dogs[features + ['stranger_aggression_score']].corr(method='spearman')
COLS = CORR.columns.to_list()
CORR = CORR.reset_index().melt(id_vars='index', value_vars=COLS)
CORR.columns = ['A', 'B', 'coef']
CORR['size'] = CORR['coef'].abs()
CORR = CORR.query('coef < 1')
CORR = CORR.sort_values(by=['A','B'], ascending=[True, False])

fig = px.scatter(CORR, x='A', y='B', size='size', color='coef', width=1200, height=1200, color_continuous_scale=px.colors.diverging.RdBu[::-1], range_color=[-1,1], hover_data={'A':True,'B':True,'coef':True,'size':False}, size_max=25)
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',   # Transparent plot background
    paper_bgcolor='rgba(30,30,30,255)',  # Transparent overall background
    xaxis=dict(showgrid=False, gridcolor='rgba(222, 222, 222, 1)', layer="below traces", autorange='reversed'),     # Hide x gridlines
    yaxis=dict(showgrid=False, gridcolor='rgba(222, 222, 222, 0.3)', layer="below traces"),
    font=dict(color="white"),
    xaxis_title='',
    yaxis_title='',
)
fig.show()

Zaujímave korelácie:
---
* Anxious a Human dependend
* Playful dogs a Human oriented
* Sociable dogs a Submissive
* Sociable dogs opposite gender a Dominant

# Chovatelia

In [ ]:
A = dogs[['dog_breeder', 'leashed_unleashed']].value_counts().reset_index()
A['leashed_unleashed'] = pd.Categorical(A['leashed_unleashed'], ordered=True, categories=['unleashed', 'partly_leashed', 'leashed'])
A = A.sort_values(by='leashed_unleashed')
A['total'] = A.groupby('dog_breeder')['count'].transform('sum')
A['percentage'] = A['count'] / A['total']
fig = px.bar(A, x='leashed_unleashed', y='percentage', color='dog_breeder', barmode='group', title='Leash usage amongst dog owners')
fig.update_layout(
    xaxis_title='',
)
fig.show()


In [ ]:
sns.violinplot(dogs, x='dog_breeder', y='Obedient', hue='dog_breeder')
plt.title('Obedience of their dog according to dog owners')

In [ ]:
sns.violinplot(dogs, x='dog_breeder', y='stranger_aggression_score', hue='dog_breeder')
plt.title('Stranger aggression amongst dog owners')

Zistili sme
---
* Chovatelia posov pouzivaju menej voditko a su si istejsi, ze ich pes viac poslucha
* V stranger_aggression_score nebol prejaveny rozdiel, cize nie je mensia sanca, ze pes zautoci na cudzieho cloveka, a tym, ze chovatel nepouziva voditko, tak to moze byt nebezpecnejsie

# Agresivita psov

In [ ]:
aggressive_columns = ['Aggressive_dogs_samegender', 'Aggressive_people', 'Aggressive_dogs_oppositegender', 'aggressiveness_dominance_score']

means = dogs.groupby('breed_group')[aggressive_columns].mean().reset_index()
means['Aggressive_columns_sum'] = means[aggressive_columns[0]] + means[aggressive_columns[1]] + means[aggressive_columns[2]]
means.sort_values(by='Aggressive_columns_sum', inplace=True)
fig = px.bar(means, y='breed_group', x=aggressive_columns[:-1], title='Dog aggression by breed')
fig.update_layout(
    xaxis_title='aggression',
    yaxis=dict(tickmode='linear'),  # Force all ticks to show
    width=1500,
    height=800,
)
fig.show()

In [ ]:
STRANGER_PHOBIAS = ['Stranger_withdraw',
 'Stranger_bark',
 'Stranger_growl',
 'Stranger_taillow',
 'Stranger_staynear',
 'Stranger_notwilling',
 'Stranger_enthusiastic',
 'Stranger_indifferent',
 'Stranger_lowposture',
 'Stranger_moveaway',
 'Stranger_suspicious']

dogs_vs_strangers = dogs[STRANGER_PHOBIAS].dropna().transpose()
sns.clustermap(dogs_vs_strangers, xticklabels=False, cmap='YlOrBr', figsize=(10,6))

In [ ]:

soc = dogs[['socialization', 'stranger_aggression_score']]
# soc['aggressivness'] = dogs[aggressive_columns[0]] + dogs[aggressive_columns[1]] + dogs[aggressive_columns[2]]
soc = soc.sort_values(by='socialization').dropna()
soc = soc.rolling(200).mean()
fig = px.line(data_frame=soc, x='socialization', y='stranger_aggression_score', title='Influence of socialization on stranger aggression', width=1500)
fig.show()

In [ ]:
display(dogs['stranger_aggression_score'].describe())

In [ ]:
A = dogs.copy()
A['Socialization_people'] = (dogs[['Socialization_men', 'Socialization_women', 'Socialization_children']].sum(axis=1) / 3).apply('round')
# A['Socialization_urban'] = (dogs[['Socialization_city', 'Socialization_public.transportation']].sum(axis=1) / 2).apply('round')
# A['Socialization_public.transportation']
fig, axe = plt.subplots(2,2, figsize=(15,10))
for i,x in enumerate(['Socialization_people', 'Socialization_dogs', 'Socialization_city', 'Socialization_public.transportation']):
    ax = sns.boxplot(A, x=x, y='stranger_aggression_score', ax=axe[i//2,i%2])
    ax.set_title(x)
    ax.set_xlabel("")
display(Markdown('**Preco je v socializacii ludmi taky skok ?**'))
print(f'Počet záznamov so socializáciou 1: {A.query('Socialization_people == 1').shape[0]}')

In [ ]:
px.scatter(dogs, x='fearfulness_score', y='stranger_aggression_score').show()

A = pd.concat([dogs, pd.cut(dogs['fearfulness_score'], np.linspace(-2, 5, 50)).rename('interval')], axis=1)

B = A.groupby('interval')['stranger_aggression_score'].mean()
axe = sns.barplot(B)

C = pd.concat([A, A.groupby('interval')['stranger_aggression_score'].transform('count').rename('CCC')], axis=1)
axe = sns.stripplot(C.loc[C['CCC'] < 10], x='interval', y='stranger_aggression_score', color='red', ax=axe)
axe.set_xticklabels('')
axe.set_xlabel('fearfulness_score')
pass

In [ ]:
x = 'stranger_aggression_score'
y = 'owner_aggression_score'
px.scatter(dogs, x=x, y=y).show()
A = dogs.copy()
A['interval'] = pd.cut(A['stranger_aggression_score'], np.linspace(-1.5, 8.2, 50))
ax = sns.pointplot(A.groupby('interval')['owner_aggression_score'].mean(), markersize=3)
ax.set_xlabel('stranger_aggression_score')
ax.set_xticklabels('')
pass

Prečo je na začiatku spád ?
---
Hypotéza:  Pes sa hanbi pred cudzimi - na majitelov si dovolia, lebo vedia, ze im nic nespravia, ale od cudzich nevedia co maju cakat.

In [ ]:
x = 'stranger_aggression_score'
y = 'owner_aggression_score'
px.scatter(dogs, x=x, y=y, color='Food_growl')

In [ ]:
def simpleline(x, y):
    return sns.lineplot(dogs.groupby(x)[y].mean(), label=x, marker='o')

CORR = dogs[features + ['stranger_aggression_score']].corr(method='spearman')
COLS = CORR.columns.to_list()
CORR = CORR.reset_index().melt(id_vars='index', value_vars=COLS)
CORR.columns = ['A', 'B', 'coef']
CORR['size'] = CORR['coef'].abs()
CORR = CORR.query('coef < 1')

F = CORR.query('A == "stranger_aggression_score" and coef >= 0.2')['B'].to_list()
F.remove('Aggressive_people')
for i,x in enumerate(F):
    simpleline(x, 'stranger_aggression_score')
plt.legend()
plt.xlabel('')
plt.ylabel('Stranger Aggression')
plt.xticks(range(1,6))

In [ ]:
F = CORR.query('A == "stranger_aggression_score" and coef <= -0.2')['B'].to_list()
for i,x in enumerate(F):
    simpleline(x, 'stranger_aggression_score')
plt.legend()
plt.ylabel('Stranger Aggression')
plt.xlabel('')
plt.xticks(range(1,6))

In [ ]:
females = dogs.query('sex == "female"')
males = dogs.query('sex == "male"')

# display(dogs.loc[:, ['Aggression_age', 'stranger_aggression_score']])
fig, axes = plt.subplots(1,2, sharex=True, sharey=True, figsize=(10,6))
# sns.scatterplot(females, x='Aggression_age', y='stranger_aggression_score', c='darkred', alpha=0.2, ax=axes[0])
sns.regplot(males, x='Aggression_age', y='stranger_aggression_score', color='darkblue', ax=axes[0], line_kws=dict(color='lightblue'), scatter_kws={'alpha':0.2}, order=2)
sns.regplot(females, x='Aggression_age', y='stranger_aggression_score', color='darkred', ax=axes[1], line_kws=dict(color='#ff7566'), scatter_kws={'alpha':0.2}, order=2)
axes[0].set_title('Males')
axes[1].set_title('Females')

# Outlayers

## Pes, ktorý sa dožil 100 rokov

In [ ]:
categories = ['Noise.sensitivity_age',
'Fearfulness_age',
'Aggression_age',
'Surface_age',
'Separation_age',
'Impulsivity_age',
'Personality_age']
ages = dogs[categories]
# display(ages.describe())
display(dogs.query('Surface_age > 100'))#.transpose().rename(columns={15370: 'Pes, ktorý sa dožil 100 rokov'}))

## Pes s vahou >100kg

In [ ]:
display(dogs.query('weight_kg > 100'))

In [ ]:
A = dogs.set_index('dog.id').query('Aggressive_people >= 4 and Affectionate_people >= 4')[['Aggressive_people', 'Affectionate_people']]
n = A.shape[0]
display(Markdown(f'<h2>{n} ľudí vyplnilo, že ich pes je 4 a viac agresívny voči ľuďom a zároveň 4 a viac prítulný k ľudom</h2>'))

# Separačná úzkosť

In [ ]:
def get_subset(values):
    return dogs.loc[:, values ]

def get_mean():
    return dogs["separation_behavior_score"].mean()

## Rozdelenie na úzkostlivé a neúzkostlivé

### Rozdelenie psov podľa podľa odpovedí majitela ohladom separačnej úzkostlivosti.

In [ ]:
def graph_sep_anx1():
    subset = get_subset(["separation_anxiety", "separation_behavior_score"])
    sns.countplot(data=subset, x="separation_anxiety", hue="separation_anxiety", legend = False)
    plt.title("Count of Separation Anxiety Cases based on owners answers")
    plt.xlabel("")
    plt.xticks([0.0, 1.0], ["Not anxious", "Anxious"])
    plt.ylabel("Count")

graph_sep_anx1()

### Rozdelenie na nadpriemerne a podpriemerne úzkostlivé

In [ ]:
def graph_sep_anx2():
    mean = get_mean()
    subset = get_subset(["separation_anxiety", "separation_behavior_score"])


    temp = []
    for i in subset["separation_behavior_score"]:
        if i<= mean:
            temp.append(0.0)
        elif i> mean:
            temp.append(1.0)
        else:
            temp.append(None)


    subset["anxious"] = temp

    sns.countplot(data=subset ,x="anxious",hue = "anxious")
    plt.title("Count of Separation Anxiety Cases Based on Average Measured Score")

    plt.xlabel("")
    plt.xticks([0.0, 1.0], ["Less Anxious Than Average", "More Anxious Than Average"])
    plt.ylabel("Count")
    plt.legend().remove()
graph_sep_anx2()

### Rozdelenie na neúzkostlivé podľa majitela, ale nadpriemerne úzkostlivé a úzkostlivé podla majiteľa ale podpriemerne úzkostlivé.

In [ ]:
def graph_sep_anx3():
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    mean = get_mean()
    subset = get_subset(["separation_anxiety", "separation_behavior_score"])
    sns.countplot(data=subset.query("separation_anxiety == 0 and separation_behavior_score>=@mean"), x="separation_anxiety",ax=axes[0])
    sns.countplot(data=subset.query("separation_anxiety == 1 and separation_behavior_score<=@mean"), x="separation_anxiety",ax=axes[0])

    axes[0].set_xlabel("")
    axes[0].set_xticks([0.0, 1.0], ["More Anxious Than Average\n(Not Anxious — Owner)", "Less Anxious Than Average\n(Anxious — Owner)"])
    axes[0].set_ylabel("Count")
    ######

    sns.countplot(data=subset.query("separation_anxiety==0"), x="separation_anxiety", color = "#69d",ax=axes[1])
    sns.countplot(data=subset.query("separation_anxiety == 0 and separation_behavior_score>@mean"), x="separation_anxiety",color = "orange",ax=axes[1])

    sns.countplot(data=subset.query("separation_anxiety==1"), x="separation_anxiety",color = "orange",ax=axes[1])
    sns.countplot(data=subset.query("separation_anxiety == 1 and separation_behavior_score<=@mean"), x="separation_anxiety",color = "#69d",ax=axes[1])

    axes[1].set_xlabel("")
    axes[1].set_xticks([0.0, 1.0], ["Not Anxious (Owner)","Anxious (Owner)" ])
    axes[1].set_ylabel("Count")

    #pomoc od chatgpt
    custom_legend = [
        Patch(facecolor="orange", label='More Anxious Than Average'),
        Patch(facecolor="#69d", label='Less Anxious Than Average')
    ]
    axes[1].legend(handles=custom_legend, title="Legend", loc='upper right')
    #koniec pomoci od chatgpt
    plt.suptitle("Owner Responses Compared to Average Separation Anxiety Level")

graph_sep_anx3()

## Hľadanie najčastejšieho a najmenej častého príznaku separačnej úzkostlivosti

### Početnosť nazbieraných hodnôt

In [ ]:
def graph_sep_anx4():
    mean = get_mean()
    columns = ['Leaving_pant', 'Leaving_vocalization', 'Leaving_salivate', 'Leaving_restless', 'Alone_destroy', 'Alone_urinate', 'Alone_vocalize', 'Alone_pant']
    subset = get_subset(["separation_anxiety", "separation_behavior_score"]+columns)


    fig, axes = plt.subplots(2, 4, figsize=(18, 10))
    axes = axes.flatten()

    for i, col in enumerate(columns):
        
        sns.countplot(x=col, data=subset, ax=axes[i],hue = col, palette = sns.dark_palette("#69d", reverse=True, as_cmap=True))
        axes[i].set_title(col.replace("_", " "))
        axes[i].set_xlabel("")
        axes[i].set_ylabel("Count")

        tick_labels = {0.0: "Never", 1.0: "Rarely", 2.0: "Sometimes", 3.0: "Often", 4.0: "Very often"}
        axes[i].set_xticks(list(tick_labels.keys()))
        axes[i].set_xticklabels(list(tick_labels.values()), rotation=15, ha='right')
        axes[i].legend().remove()
    plt.tight_layout()
    plt.show()
graph_sep_anx4()

### Priemery nazbieraných hodnôt

In [ ]:
def graph_sep_anx5():
    mean = get_mean()
    columns = ['Leaving_pant', 'Leaving_vocalization', 'Leaving_salivate', 'Leaving_restless', 'Alone_destroy', 'Alone_urinate', 'Alone_vocalize', 'Alone_pant']
    subset = get_subset(["separation_anxiety", "separation_behavior_score"]+columns)

    mean_values, symptoms = zip(*sorted([(subset[i].mean(),i.replace("_", " ")) for i in columns], key=lambda x: x[0])[::-1])
    plt.figure(figsize=(10, 6))
    palette = sns.dark_palette("#69d", reverse=True, n_colors=len(mean_values))

    plt.bar(symptoms, mean_values, color = palette)

    plt.xticks(rotation=15, ha='right')
    plt.xlabel('Symptoms of separation anxiety')

    ticks = plt.yticks()[0]   
    plt.yticks(ticks[::2])  
    plt.ylabel('Means')

    plt.title('Means of symptoms of separation anxiety')

graph_sep_anx5()

## Zisťujeme najúzkostlivejšiu skupinu plemien psov

In [ ]:
def graph_sep_anx6():
    mean = get_mean()
    subset = get_subset(["separation_behavior_score",  "breed_group"])

    a = subset.groupby("breed_group")["separation_behavior_score"].mean().sort_values(ascending=False)
    plt.figure(figsize=(14, 8))

    palette = sns.dark_palette("#69d", n_colors=len(a))[::-1]
    bars = plt.bar(a.index, a.values, color = palette)

    plt.axhline(y=mean, color='red', linestyle='-', linewidth=1)
    plt.text(len(a) - 0.5, mean + 0.01, 'mean', color='red', fontsize=8)
    plt.xticks([i for i in a.index], [i.replace("_", " ") for i in a.index],rotation=40, ha='right',fontsize = 8)
    plt.title('Average Separation Behavior Score by Breed Group')
    plt.ylabel('Mean Separation Score')
    plt.xlabel('Breed Group')

    plt.show()

graph_sep_anx6()

## Separačná úzkosť vzhľadom na niektoré informácie o pôvode psov.

In [ ]:
def graph_sep_anx7():
    mean = get_mean()
    subset = get_subset(["separation_behavior_score","obtained_from","alone_time"])

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))

    #alone time
    alone = subset.groupby("alone_time")["separation_behavior_score"].mean().sort_values(ascending=False)
    palette = sns.dark_palette("#69d", n_colors=len(alone))[::-1]

    axes[0].bar(alone.index, alone.values, color = palette )

    axes[0].set_title('Average Separation Behavior Score by Time Alone')
    axes[0].set_ylabel('Average Separation Score')
    axes[0].set_xlabel('Time Alone')
    axes[0].set_xticks(range(len(alone)))
    axes[0].set_xticklabels([i.replace("_", " ") for i in alone.index], rotation=40, ha='right')

    #obtained_from

    obtained = subset.groupby("obtained_from")["separation_behavior_score"].mean().sort_values(ascending=False)
    palette = sns.dark_palette("#69d", n_colors=len(obtained))[::-1]

    axes[1].bar(obtained.index, obtained.values, color = palette, )
    axes[1].set_title('Average Separation Behavior Score by Method of Acquisition')
    axes[1].set_ylabel('Average Separation Behavior Score')
    axes[1].set_xlabel('Method of Acquisition')
    axes[1].set_xticks(range(len(obtained)))
    axes[1].set_xticklabels([i.replace("_", " ") for i in obtained.index], ha='right')


graph_sep_anx7()

Uskostlivosť vzhľadom na vek

In [ ]:
def graph_sep_anx10():
    plt.figure(figsize=(10, 6))
    mean = get_mean()
    subset = get_subset(["separation_behavior_score", "Separation_age"])

    sns.scatterplot(
        subset,
        x="Separation_age",
        y="separation_behavior_score",
    )

    plt.xlabel("Age Of Separation")
    plt.ylabel("Separation Behavior Score")
    plt.title("Separation Behavior Score By Separation Age")
    plt.axhline(y=mean, color='red', linestyle='-', linewidth=1)
    plt.text(max(subset["Separation_age"]) - 0.1, mean + 0.01, 'mean', color='red', fontsize=10)
    plt.show()

graph_sep_anx10()

# Načítanie dát znova za účelom nemixovania spracovávaní datasetu

In [ ]:
dogs = pd.read_csv("https://raw.githubusercontent.com/Retr-1/AnalyzaPsy/refs/heads/master/psy_data.csv", header=0).set_index("dog.id")
dog_info = pd.read_csv("https://raw.githubusercontent.com/Retr-1/AnalyzaPsy/refs/heads/master/psy_info.csv", header=0)
casti = ['Demographic information', 'Environment and background', 'Personality', 
     'Noise sensitivity', 'Fearfulness', 'Aggression', 
     'Fear of surfaces and heights', 'Separation anxiety', 'Impulsivity/ inattention']
zoz = []
for i in dog_info['SECTION']:
    if i in casti:
        slovo = i
    zoz.append(slovo)
dog_info['SECTION'] = pd.Series(zoz)
dog_info = dog_info.set_index('SECTION').drop('Unnamed: 0', axis=1)

## Tuto budu zoznamy jednotlivych casti, a urobene casti tabulky podla kategorie


In [ ]:
spojite_vsetky = dog_info[dog_info['COLUMN VALUES'].str.contains('continuous', na=False)]
spojite = list(spojite_vsetky.query('SECTION != "Demographic information"')['COLUMN NAME'])
spojite_set = set(spojite)


dogs_male = dogs.query("sex=='male'")
dogs_female = dogs.query("sex=='female'")

dogs_demographic = dogs.transpose().iloc[0:11].transpose()
dogs_environment = dogs.transpose().iloc[11:42].transpose()
dogs_personality = dogs.transpose().iloc[42:112].transpose()
dogs_noise_sensitivity = dogs.transpose().iloc[112:154].transpose()
dogs_fearfullness = dogs.transpose().iloc[154:189].transpose()
dogs_aggression = dogs.transpose().iloc[189:214].transpose()
dogs_fear_surfaces = dogs.transpose().iloc[214:222].transpose()
dogs_separation_anxiety = dogs.transpose().iloc[222:232].transpose()
dogs_inattention = dogs.transpose().iloc[232:250].transpose()


demographic = list(dogs_demographic.columns)
environment = list(dogs_environment.columns)
personality = list(dogs_personality.columns)
noise_sensitivity = list(dogs_noise_sensitivity.columns)
fearfullness = list(dogs_fearfullness.columns)
aggression = list(dogs_aggression.columns)
fear_surfaces = list(dogs_fear_surfaces.columns)
separation_anxiety = list(dogs_separation_anxiety.columns)
inattention = list(dogs_inattention.columns)

demographic_continuous = list(spojite_set & set(demographic))
environment_continuous = list(spojite_set & set(environment))
personality_continuous = list(spojite_set & set(personality))
noise_sensitivity_continuous = list(spojite_set & set(noise_sensitivity))
fearfullness_continuous = list(spojite_set & set(fearfullness))
aggression_continuous = list(spojite_set & set(aggression))
fear_surfaces_continuous = list(spojite_set & set(fear_surfaces))
separation_anxiety_continuous = list(spojite_set & set(separation_anxiety))
inattention_continuous = list(spojite_set & set(inattention))

demographic_discrete = list(set(demographic) - spojite_set)
environment_discrete = list(set(environment) - spojite_set)
personality_discrete = list(set(personality) - spojite_set)
noise_sensitivity_discrete = list(set(noise_sensitivity) - spojite_set)
fearfullness_discrete = list(set(fearfullness) - spojite_set)
aggression_discrete = list(set(aggression) - spojite_set)
fear_surfaces_discrete = list(set(fear_surfaces) - spojite_set)
separation_anxiety_discrete = list(set(separation_anxiety) - spojite_set)
inattention_discrete = list(set(inattention) - spojite_set)

nespojite = environment_discrete+personality_discrete+noise_sensitivity_discrete+fearfullness_discrete+aggression_discrete+fear_surfaces_discrete+separation_anxiety_discrete+inattention_discrete
nespojite = list(set(nespojite)-set(environment))

# Explorativna analyza

In [ ]:
breeds = list(dogs['breed_group'].unique())
breed_count = len(breeds)
male_count = len(dogs.query("sex=='male'"))
female_count = len(dogs.query("sex=='female'"))

# print('plemena:', breeds)
print('mnozstvo samcov:', male_count)
print('mnozstvo samic:', female_count)
print('nozstvo plemien:', breed_count)


figure, axes = plt.subplots(1, 1, figsize=(13, 10))
sns.violinplot(data=dogs[spojite], orient='h', ax=axes)
axes.set_title('Distribution of continuous scores')


## Agresivita
#### samce vs samice

In [ ]:

figure, axes = plt.subplots(1, 1, figsize=(6, 5), sharey=True)

# popisy = {
#     'Aggressive_dogs_samegender': 'Voči psom rovnakého pohlavia',
#     'Aggressive_dogs_oppositegender': 'Voči psom opačného pohlavia',
#     'Aggressive_people': 'Voči ľuďom',
#     'aggressiveness_dominance_score': 'Dominancia / Agresivita',
#     'stranger_aggression_score': 'Agresivita voči cudzim',
#     'owner_aggression_score': 'Agresivita voči majiteľovi',
#     'dog_aggression_score': 'Agresivita voči psom',
#     'barking_score': 'Brechanie'
# }

# First plot
agresivita_gender = dogs.groupby('sex')[['Aggressive_dogs_samegender', 'Aggressive_dogs_oppositegender', 'Aggressive_people']].mean().reset_index().melt(id_vars='sex')
agresivita_gender['variable'] = agresivita_gender['variable']#.replace(popisy)

sns.barplot(agresivita_gender, y='variable', x='value', hue='sex', palette={'male':'blue', 'female':'violet'}, ax=axes, orient='h')

axes.set_title('Average Aggression', fontsize=14)
axes.set_xlabel('Value', fontsize=12)
axes.set_ylabel('', fontsize=12)
axes.legend(title='Sex')
plt.tight_layout()
plt.show()

# Second plot
figure, axes = plt.subplots(1, 1)
plt.xticks(ha='right') 
agresivita_gender = dogs.groupby('sex')[['aggressiveness_dominance_score', 'stranger_aggression_score', 'owner_aggression_score', 'dog_aggression_score', 'barking_score']].mean().reset_index().melt(id_vars='sex')
agresivita_gender['variable'] = agresivita_gender['variable']#.replace(popisy)

sns.barplot(agresivita_gender, x='variable', y='value', hue='sex', palette={'male':'blue', 'female':'violet'})
axes.tick_params(axis='x', rotation=25)

axes.set_title('Average Aggression and Barking', fontsize=14)
axes.set_ylabel('Value', fontsize=12)
axes.set_xlabel('', fontsize=12)
axes.set_ylim(-0.25, 0.25)
axes.legend(title='Sex')
plt.tight_layout()
plt.show()


# Kastracia
zistovanie najrozdielnejsich vlastnosti v ramci kastracie

In [ ]:
print('pocet sterilizovanych:', len([x for x in dogs['sterilization_status'] if x == "intact"]))

pocet = 10

kastracia_male = dogs.query('sex=="male"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_male.columns.name = None
kastracia_male["personality_trait"] = kastracia_male.index

kastracia_rozdiely_male_nespojite = kastracia_male.loc[nespojite, :].copy(deep=True)
kastracia_rozdiely_male_nespojite["diff"] = abs(kastracia_rozdiely_male_nespojite["sterilized"] - kastracia_rozdiely_male_nespojite["intact"])
kastracia_rozdiely_male_nespojite["graf_diff"] = (kastracia_rozdiely_male_nespojite["sterilized"] - kastracia_rozdiely_male_nespojite["intact"])
kastracia_rozdiely_male_nespojite["positive"] = (kastracia_rozdiely_male_nespojite["graf_diff"]>0)
kastracia_rozdiely_male_nespojite = kastracia_rozdiely_male_nespojite.sort_values("graf_diff", key=abs, ascending=False).head(pocet)

index_male_nespojite = list(kastracia_rozdiely_male_nespojite.sort_values('diff', ascending=False).head(pocet).index)


kastracia_rozdiely_male_spojite = kastracia_male.loc[spojite, :].copy(deep=True)
kastracia_rozdiely_male_spojite["diff"] = abs(kastracia_rozdiely_male_spojite["sterilized"] - kastracia_rozdiely_male_spojite["intact"])
kastracia_rozdiely_male_spojite["graf_diff"] = (kastracia_rozdiely_male_spojite["sterilized"] - kastracia_rozdiely_male_spojite["intact"])
kastracia_rozdiely_male_spojite["positive"] = (kastracia_rozdiely_male_spojite["graf_diff"]>0)
kastracia_rozdiely_male_spojite = kastracia_rozdiely_male_spojite.sort_values("graf_diff", key=abs, ascending=False).head(pocet)

index_male_spojite= list(kastracia_rozdiely_male_spojite.sort_values('diff', ascending=False).head(pocet).index)

###################################################################################################################################

kastracia_female = dogs.query('sex=="female"').groupby("sterilization_status").mean(numeric_only=True).transpose().iloc[8:]
kastracia_female.columns.name = None
kastracia_female["personality_trait"] = kastracia_female.index

kastracia_rozdiely_female_nespojite = kastracia_female.loc[nespojite, :].copy(deep=True)
kastracia_rozdiely_female_nespojite["diff"] = abs(kastracia_rozdiely_female_nespojite["sterilized"] - kastracia_rozdiely_female_nespojite["intact"])
kastracia_rozdiely_female_nespojite["graf_diff"] = (kastracia_rozdiely_female_nespojite["sterilized"] - kastracia_rozdiely_female_nespojite["intact"])
kastracia_rozdiely_female_nespojite["positive"] = (kastracia_rozdiely_female_nespojite["graf_diff"]>0)
kastracia_rozdiely_female_nespojite = kastracia_rozdiely_female_nespojite.sort_values("graf_diff", key=abs, ascending=False).head(pocet)
index_female_nespojite = list(kastracia_rozdiely_female_nespojite.sort_values('diff', ascending=False).head(pocet).index)


kastracia_rozdiely_female_spojite = kastracia_female.loc[spojite, :].copy(deep=True)
kastracia_rozdiely_female_spojite["diff"] = abs(kastracia_rozdiely_female_spojite["sterilized"] - kastracia_rozdiely_female_spojite["intact"])
kastracia_rozdiely_female_spojite["graf_diff"] = (kastracia_rozdiely_female_spojite["sterilized"] - kastracia_rozdiely_female_spojite["intact"])
kastracia_rozdiely_female_spojite["positive"] = (kastracia_rozdiely_female_spojite["graf_diff"]>0)
kastracia_rozdiely_female_spojite = kastracia_rozdiely_female_spojite.sort_values("graf_diff", key=abs, ascending=False).head(pocet)
index_female_spojite= list(kastracia_rozdiely_female_spojite.sort_values('diff', ascending=False).head(pocet).index)


## Kastracia pokracovanie - Grafy
vygrafovanie zistenych vlastnosti

In [ ]:

dogs_melted_male_spojite = kastracia_male.loc[index_male_spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_female_spojite = kastracia_female.loc[index_female_spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

palette = {'intact': 'dodgerblue', 'sterilized': 'tomato'}

figure, axes = plt.subplots(1, 2, figsize=(18, 6), sharey=True)

sns.barplot(
    data=dogs_melted_male_spojite, x=dogs_melted_male_spojite.index, y="value", hue="sterilization_status",
    ax=axes[0], palette=palette, dodge=0.6, width=0.4, edgecolor='black', linewidth=0.5
)
axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("Male")
axes[0].set_xlabel('')

sns.barplot(
    data=dogs_melted_female_spojite, x=dogs_melted_female_spojite.index, y="value", hue="sterilization_status",
    ax=axes[1], palette=palette, dodge=0.6, width=0.4, edgecolor='black', linewidth=0.5
)
axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("Female")
axes[1].set_xlabel('')

plt.suptitle("Traits with the Greatest Variance by Sterilization Status – Continuous", fontsize=16)
plt.tight_layout()
plt.show()


###########################################

dogs_melted_male_nespojite = kastracia_male.loc[index_male_nespojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_female_nespojite = kastracia_female.loc[index_female_nespojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

figure, axes = plt.subplots(1, 2, figsize=(18, 6), sharey=True)
for ax in axes:
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)

sns.barplot(data=dogs_melted_male_nespojite, x=dogs_melted_male_nespojite.index, y="value", hue="sterilization_status", ax=axes[0], palette=palette, dodge=0.6, width=0.5, edgecolor='black', linewidth=0.5)
plt.xticks(ha='right') 
axes[0].tick_params(axis='x', rotation=30)
axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
axes[0].set_xlabel('')
axes[0].set_title("Male")
sns.barplot(data=dogs_melted_female_nespojite, x=dogs_melted_female_nespojite.index, y="value", hue="sterilization_status", ax=axes[1], palette=palette, dodge=0.6, width=0.5, edgecolor='black', linewidth=0.5)
axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')
axes[1].tick_params(axis='x', rotation=30)
axes[1].set_xlabel('')
axes[1].set_title("Female")
plt.suptitle("Traits with the Greatest Variance by Sterilization Status - discrete")


#  Kastracia Rozdiely

In [ ]:
# display(kastracia_rozdiely_male_spojite)
palette = {False: 'green', True: 'red'}

figure, axes = plt.subplots(2, 2, figsize=(12, 12), sharey=True)

sns.barplot(data=kastracia_rozdiely_male_nespojite, x=kastracia_rozdiely_male_nespojite.index, y="diff", hue='positive', ax=axes[0][0], palette=palette)
axes[0][0].set_title("Males – Categorical Variables")

sns.barplot(data=kastracia_rozdiely_female_nespojite, x=kastracia_rozdiely_female_nespojite.index, y="diff", hue='positive', ax=axes[0][1], palette=palette)
axes[0][1].set_title("Females – Categorical Variables")

sns.barplot(data=kastracia_rozdiely_male_spojite, x=kastracia_rozdiely_male_spojite.index, y="diff", hue='positive', ax=axes[1][0], palette=palette)
axes[1][0].set_title("Males – Continuous Variables")

sns.barplot(data=kastracia_rozdiely_female_spojite, x=kastracia_rozdiely_female_spojite.index, y="diff", hue='positive', ax=axes[1][1], palette=palette)
axes[1][1].set_title("Females – Continuous Variables")

handles, labels = axes[0][0].get_legend_handles_labels()
for ax in axes.flat:
    ax.set_xlabel("")
    ax.set_ylabel("difference")
    ax.legend(handles, ['Intact', 'Sterilized'], title='Which group scores higher')
    ax.tick_params(axis='x', labelrotation=30)
    ax.set_xticklabels(ax.get_xticklabels(), ha='right')

plt.suptitle("Difference")
plt.tight_layout()
plt.show()



## Kastracia dalsia
planujem urobit ze zoberiem spojite premenne z jednotlivych casti dotaznika
a porovnam s ktorou suvisi kastrovanie najviac (nemam nicim podlozene, ale pride mi ze pri jednotlivych otazkach to moze byt aj trochu nahodne)

In [ ]:
# dogs_melted_spojite_male = kastracia_male.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
# dogs_melted_spojite_female = kastracia_female.loc[spojite, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

# palette = {'intact': 'green', 'sterilized': 'red'}

# figure, axes = plt.subplots(1, 2, figsize=(14, 7))
# sns.barplot(data=dogs_melted_spojite_male, y=dogs_melted_spojite_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h', palette=palette)
# # axes[0].tick_params(axis='y', rotation=30)
# axes[0].set_title("male")
# # axes[0].set_xticklabels(axes[0].get_xticklabels(), ha='right')
# sns.barplot(data=dogs_melted_spojite_female, y=dogs_melted_spojite_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h', palette=palette)
# # axes[1].tick_params(axis='y', rotation=30)
# axes[1].set_title("female")
# # axes[1].set_xticklabels(axes[1].get_xticklabels(), ha='right')

## Kastracia a agresivita
porovnam podobnym bargrafom agresivitu

In [ ]:
dogs_melted_aggression_male = kastracia_male.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")
dogs_melted_aggression_female = kastracia_female.loc[aggression_discrete, :].melt(id_vars=["personality_trait"], var_name='sterilization_status').set_index("personality_trait")

# palette = {'intact': 'blue', 'sterilized': 'red'}


figure, axes = plt.subplots(1, 2, figsize=(9, 8), sharey=True)
sns.barplot(data=dogs_melted_aggression_male, y=dogs_melted_aggression_male.index, x="value", hue="sterilization_status", ax=axes[0], orient='h', width=0.6)
# axes[0].tick_params(axis='x', rotation=30)
axes[0].set_title("male")
axes[0].set_ylabel("")
sns.barplot(data=dogs_melted_aggression_female, y=dogs_melted_aggression_female.index, x="value", hue="sterilization_status", ax=axes[1], orient='h', width=0.6)
# axes[1].tick_params(axis='x', rotation=30)
axes[1].set_title("female")

plt.suptitle("Difference in Aggresion traits with Sterilization")


### Tuto je agresivita psov
##### ku psom, majitelom a cudzncom
podla veku 

In [ ]:
psy1 = dogs.copy(deep=True)
psy1["Aggression_age"] = psy1["Aggression_age"].round(1)

psy1 = psy1.groupby("Aggression_age")[aggression_continuous].mean()

fig, axes = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="Aggression_age", ax=axes, binwidth=0.1)
axes.set_title("Histogram of Questionnaire Age", fontsize=13)
axes.set_xlabel("Age (years)", fontsize=12)
axes.set_ylabel("Number of Dogs", fontsize=12)
plt.tight_layout()
plt.show()

fig, axes = plt.subplots(1, 3, figsize=(16, 5), sharey=True)

sns.scatterplot(x=psy1.index, y=psy1["stranger_aggression_score"], ax=axes[0])
axes[0].set_title("Toward Strangers")
axes[0].set_xlabel("Age (years)")
axes[0].set_ylabel("Aggression")

sns.scatterplot(x=psy1.index, y=psy1["owner_aggression_score"], ax=axes[1])
axes[1].set_title("Toward Owner")
axes[1].set_xlabel("Age (years)")

sns.scatterplot(x=psy1.index, y=psy1["dog_aggression_score"], ax=axes[2])
axes[2].set_title("Toward Other Dogs")
axes[2].set_xlabel("Age (years)")

plt.suptitle("Types of Aggression by Dog Age", fontsize=14)
plt.tight_layout()
plt.show()



### Teraz spravim podobne impulzivitu a nepozornost
nasiel som jemnu zavislost, impulzivita stupa a vrcholi okolo 1 roka, potom pomaly klesa, nepozornost je na tom velmi podobne



In [ ]:
psy2 = dogs.copy(deep=True)
psy2["Impulsivity_age"] = psy2["Impulsivity_age"].round(1)
psy2 = psy2.groupby("Impulsivity_age")[inattention_continuous].mean()

ax = sns.histplot(data=dogs, x="Impulsivity_age", binwidth=0.1)
ax.set_title("Histogram of Questionnaire Age for Impulsivity")
ax.set_xlabel("Age (years)")
ax.set_ylabel("Number of Dogs")

fig, axes = plt.subplots(1, 2, figsize=(16, 5), sharey=True)

sns.scatterplot(x=psy2.index, y=psy2["inattention_score"], ax=axes[0])
sns.regplot(x=psy2.index, y="inattention_score", data=psy2, ax=axes[0], scatter=True, line_kws={"color": "red"})
axes[0].set_title("Inattention")
axes[0].set_xlabel("Age (years)")
axes[0].set_ylabel("Score")

sns.scatterplot(x=psy2.index, y=psy2["impulsivity_score"], ax=axes[1])
sns.regplot(x=psy2.index, y="impulsivity_score", data=psy2, ax=axes[1], scatter=True, line_kws={"color": "red"})
axes[1].set_title("Impulsivity")
axes[1].set_xlabel("Age (years)")
axes[1].set_ylabel("Score")

plt.suptitle("Inattention and Impulsivity by Dog Age", fontsize=14)
plt.tight_layout()
plt.show()


### Teraz spravim podobne strach
pri strachu ale nevidim ziadnu suvislost




In [ ]:
psy3 = dogs.copy(deep=True)
psy3["Fearfulness_age"] = psy3["Fearfulness_age"].round(1)
psy3 = psy3.groupby("Fearfulness_age")[fearfullness_continuous].mean()

# Histogram of age at the time of filling out the fearfulness questionnaire
ax = sns.histplot(data=dogs, x="Fearfulness_age")
ax.set_title("Histogram of Questionnaire Age for Fearfulness")
ax.set_xlabel("Age (years)")
ax.set_ylabel("Number of Dogs")

# Scatter plot of fearfulness score by age
figure, axes = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.scatterplot(x=psy3.index, y="fearfulness_score", data=psy3, ax=axes)

axes.set_title("Fearfulness by Age")
axes.set_xlabel("Age (years)")
axes.set_ylabel("Fearfulness Score")

plt.tight_layout()
plt.show()



### Priemerne hmotnosti jednotlivych plemien barplot


In [ ]:

psy_hmostnost_max = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).head()
psy_hmostnost_min = pd.DataFrame(dogs.groupby('breed_group')['weight_kg'].mean().sort_values()).tail()

figure, axes = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
sns.barplot(data=psy_hmostnost_max, x=psy_hmostnost_max.index, y='weight_kg', ax=axes[0])
axes[0].tick_params(axis='x', rotation=45)
sns.barplot(data=psy_hmostnost_min, x=psy_hmostnost_min.index, y='weight_kg', ax=axes[1])
axes[1].tick_params(axis='x', rotation=45)


pocty_psov = pd.DataFrame(dogs.groupby("breed_group").count().sort_values(by="sex"))
# figure, axes1 = plt.subplots(1, 1)
# display(pocty_psov)
# sns.histplot(data=pocty_psov, ax=axes1)

### Teraz Spravim Agresivitu vs Strach

In [ ]:
psy4 = dogs.copy(deep=True)
psy4["fearfulness_score"] = psy4["fearfulness_score"].round(1)
psy4 = psy4.groupby("fearfulness_score")[aggression_continuous].mean()

# Histogram of fearfulness score
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="fearfulness_score", ax=ax, binwidth=0.1)
ax.set_title("Histogram of Fearfulness Scores")
ax.set_xlabel("Fearfulness Score")
ax.set_ylabel("Number of Dogs")
plt.tight_layout()
plt.show()

# Scatter plots: aggression vs. fear
fig, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=True)

sns.scatterplot(x=psy4.index, y=psy4["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Stranger Aggression")
axes[0][0].set_xlabel("Fearfulness Score")
axes[0][0].set_ylabel("Aggression")

sns.scatterplot(x=psy4.index, y=psy4["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Owner Aggression")
axes[0][1].set_xlabel("Fearfulness Score")
axes[0][1].set_ylabel("Aggression")

sns.scatterplot(x=psy4.index, y=psy4["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Dog Aggression")
axes[1][0].set_xlabel("Fearfulness Score")
axes[1][0].set_ylabel("Aggression")

sns.scatterplot(x=psy4.index, y=psy4["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Barking")
axes[1][1].set_xlabel("Fearfulness Score")
axes[1][1].set_ylabel("Score")

plt.suptitle("Relationship Between Fearfulness and Aggression/Barking", fontsize=14)
plt.tight_layout()
plt.show()


### Citlivost na Hluk vs Agresivita

In [ ]:
psy5 = dogs.copy(deep=True)
psy5["noise_sensitivity_score"] = psy5["noise_sensitivity_score"].round(1)
psy5 = psy5.groupby("noise_sensitivity_score")[aggression_continuous].mean()

# Histogram citlivosti na hluk
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="noise_sensitivity_score", binwidth=0.1, ax=ax)
ax.set_title("Histogram citlivosti na hluk")
ax.set_xlabel("Skóre citlivosti na hluk")
ax.set_ylabel("Počet psov")
plt.tight_layout()
plt.show()

# Scatter ploty agresivity vs. citlivosť na hluk
fig, axes = plt.subplots(2, 2, figsize=(13, 9), sharey=False)

sns.scatterplot(x=psy5.index, y=psy5["stranger_aggression_score"], ax=axes[0][0])
axes[0][0].set_title("Agresia voči cudzincom podľa citlivosti na hluk")
axes[0][0].set_xlabel("Skóre citlivosti na hluk")
axes[0][0].set_ylabel("Agresia")

sns.scatterplot(x=psy5.index, y=psy5["owner_aggression_score"], ax=axes[0][1])
axes[0][1].set_title("Agresia voči majiteľovi podľa citlivosti na hluk")
axes[0][1].set_xlabel("Skóre citlivosti na hluk")
axes[0][1].set_ylabel("Agresia")

sns.scatterplot(x=psy5.index, y=psy5["dog_aggression_score"], ax=axes[1][0])
axes[1][0].set_title("Agresia voči psom podľa citlivosti na hluk")
axes[1][0].set_xlabel("Skóre citlivosti na hluk")
axes[1][0].set_ylabel("Agresia")
# axes[1][0].set_ylim(-1, 3)

sns.scatterplot(x=psy5.index, y=psy5["barking_score"], ax=axes[1][1])
axes[1][1].set_title("Štekot podľa citlivosti na hluk")
axes[1][1].set_xlabel("Skóre citlivosti na hluk")
axes[1][1].set_ylabel("Štekot")

plt.suptitle("Vzťah medzi citlivosťou na hluk a agresivitou/štekaním", fontsize=14)
plt.tight_layout()
plt.show()

### Teraz strach vs citlivost na hluk

In [ ]:
psy6 = dogs.copy(deep=True)
psy6["noise_sensitivity_score"] = psy6["noise_sensitivity_score"].round(1)

psy6 = psy6.groupby("noise_sensitivity_score")[["fearfulness_score"]].mean()

# Histogram of noise sensitivity
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.histplot(data=dogs, x="noise_sensitivity_score", binwidth=0.1, ax=ax)
ax.set_title("Histogram of Noise Sensitivity")
ax.set_xlabel("Noise Sensitivity Score")
ax.set_ylabel("Number of Dogs")
plt.tight_layout()
plt.show()

# Scatter plot: Fearfulness vs. Noise Sensitivity
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
sns.scatterplot(x=psy6.index, y=psy6["fearfulness_score"], ax=ax)
ax.set_title("Fearfulness vs. Noise Sensitivity")
ax.set_xlabel("Noise Sensitivity Score")
ax.set_ylabel("Fearfulness Score")


# Environment

In [ ]:

order=['under_1_hour', '1-2_hours', '2-3_hours', 'over_3_hours']

psy7 = dogs.copy(deep=True)
psy7 = psy7.groupby("daily_exercise").agg('mean', numeric_only=True)[personality]
psy7 = psy7.loc[order, :]
# display(psy7)

figure, ax = plt.subplots(1, 1, figsize=(6, 4), sharey=True)
sns.countplot(data=dogs, x="daily_exercise", ax=ax, order=order)
ax.tick_params(axis='x', rotation=25)

# daily_exercise
zaujimave = ['Fearful_dogs', 'Playful_alone', 'Playful_people', 'Playful_dogs', 'Anxious', 'Lazy', 'Intelligent', 'Sociable_dogs_samegender']
dlzka = 6
figure, axes = plt.subplots(2, 4, figsize=(16, 7), sharex=True)
plt.xticks(ha='right') 
for i in range(8):
       sns.barplot(x=psy7.index, y=psy7[zaujimave[i]], ax=axes[i//4][i%4])
       # axes[i//4][i%4].set_ylim(0, 5)
       axes[i//4][i%4].tick_params(rotation=30)


plt.suptitle("Positive effect of daily exercise")
